In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Import Libraries**

In [2]:
import numpy as np
import pandas as pd
import nltk

from sklearn.preprocessing import OneHotEncoder as oneHot
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')

2024-04-09 08:15:37.145578: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 08:15:37.145727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 08:15:37.323823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Read Data**

In [3]:
#Read the train and test datasets with column names as target and text
train= pd.read_csv('/kaggle/input/atis-airlinetravelinformationsystem/atis_intents_train.csv',
                       names= ["target", "text"])

test= pd.read_csv('/kaggle/input/atis-airlinetravelinformationsystem/atis_intents_test.csv',
                       names= ["target", "text"])

In [4]:
train.shape

(4834, 2)

In [5]:
test.shape

(800, 2)

In [6]:
train.head(10) #Get Top 10 rows from train dataset

,target,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...
5,atis_flight,i need a flight tomorrow from columbus to min...
6,atis_aircraft,what kind of aircraft is used on a flight fro...
7,atis_flight,show me the flights from pittsburgh to los an...
8,atis_flight,all flights from boston to washington
9,atis_ground_service,what kind of ground transportation is availab...


In [7]:
train['target'].value_counts() #Get counts of different types of target variable in train data. We will not be using this anywhere but it is just for the overview of the data.

target
atis_flight            3666
atis_airfare            423
atis_ground_service     255
atis_airline            157
atis_abbreviation       147
atis_aircraft            81
atis_flight_time         54
atis_quantity            51
Name: count, dtype: int64

## **Preprocessing**

In [8]:
encode_target= oneHot().fit(np.array(train.target).reshape(-1,1)) #We perform one hot encoding on the target variable to convert into a matrix of 0s and 1s.

In [9]:
train_target_encoded= encode_target.transform(np.array(train.target).reshape(-1,1)).toarray()
test_target_encoded= encode_target.transform(np.array(test.target).reshape(-1,1)).toarray()

In [10]:
train_target_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
test_target_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
train["text"]= train.text.map(lambda l: l.lower())
test["text"]= test.text.map(lambda l: l.lower())

In [13]:
train.head()

,target,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [14]:
test.head()

,target,text
0,atis_flight,i would like to find a flight from charlotte ...
1,atis_airfare,on april first i need a ticket from tacoma to...
2,atis_flight,on april first i need a flight going from pho...
3,atis_flight,i would like a flight traveling one way from ...
4,atis_flight,i would like a flight from orlando to salt la...


In [15]:
train["text"]= train.text.map(word_tokenize)
test["text"]= test.text.map(word_tokenize)

In [16]:
#Output of the above exercise looks like this
train["text"]

0       [i, want, to, fly, from, boston, at, 838, am, ...
1       [what, flights, are, available, from, pittsbur...
2       [what, is, the, arrival, time, in, san, franci...
3          [cheapest, airfare, from, tacoma, to, orlando]
4       [round, trip, fares, from, pittsburgh, to, phi...
                              ...                        
4829    [what, is, the, airfare, for, flights, from, d...
4830    [do, you, have, any, flights, from, denver, to...
4831    [which, airlines, fly, into, and, out, of, den...
4832    [does, continental, fly, from, boston, to, san...
4833    [is, there, a, delta, flight, from, denver, to...
Name: text, Length: 4834, dtype: object

In [17]:
def clean_data_rm_stop(strings, stop_list):
    sw= [str for str in strings if str not in stop_list]
    return sw

stop_words= stopwords.words("english")
rm_punc_stop= list(set(punctuation))+ stop_words #Remove punctuation and stop words

train["text"]= train.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))
test["text"]= test.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))

In [18]:
def normalize(text):
    return " ".join(text)

#We use PorterStemmer function from nltk.stem library.
stem_func= PorterStemmer()

train["text"]= train.text.map(lambda s: [stem_func.stem(x) for x in s])
train["text"]= train.text.apply(normalize)

test["text"]= test.text.map(lambda s: [stem_func.stem(x) for x in s])
test["text"]= test.text.apply(normalize)

In [19]:
train.head()

,target,text
0,atis_flight,want fli boston 838 arriv denver 1110 morn
1,atis_flight,flight avail pittsburgh baltimor thursday morn
2,atis_flight_time,arriv time san francisco 755 flight leav washi...
3,atis_airfare,cheapest airfar tacoma orlando
4,atis_airfare,round trip fare pittsburgh philadelphia 1000 d...


In [20]:
# We use Tokenizer from tensorflow.keras.preprocessing.text library
num_words=10000
text_tokenizer= Tokenizer(num_words)
text_tokenizer.fit_on_texts(train.text) #fit_on_texts - creates the vocabulary index based on word frequency.

tokenized_train_data= text_tokenizer.texts_to_sequences(train.text) #Converting texts to sequences
tokenized_test_data= text_tokenizer.texts_to_sequences(test.text)

In [21]:
tokenized_train_data

[[42, 19, 2, 296, 17, 5, 376, 21],
 [1, 37, 8, 10, 45, 21],
 [17, 102, 4, 6, 377, 1, 12, 18],
 [32, 212, 142, 91],
 [36, 30, 16, 8, 11, 180, 134],
 [33, 1, 98, 174, 138],
 [164, 99, 108, 1, 100, 9],
 [3, 1, 8, 103, 104, 45],
 [1, 2, 18],
 [164, 27, 29, 37, 5],
 [3, 1, 9, 4, 6],
 [3, 1, 4, 78, 67, 43, 87],
 [32, 1, 2, 225],
 [1, 10, 113, 22],
 [3, 24, 31, 16, 2, 5],
 [3, 27, 29, 5],
 [1, 5, 8, 12, 113, 22, 132, 22],
 [33, 55, 1, 52, 12, 10, 9, 9, 2, 2, 10],
 [20, 41, 1, 2, 8, 45, 79, 204],
 [23, 14, 19, 5, 8, 53, 13],
 [3, 1, 4, 78, 67],
 [20, 15, 24, 31, 1, 53, 5, 10],
 [164, 190, 108, 39, 13],
 [28, 14, 55, 109, 5, 8, 7],
 [28, 14, 136, 1, 7, 5],
 [13, 89, 5, 8, 7],
 [3, 1, 2, 8, 25, 79, 204, 12, 2, 181, 69, 22],
 [7, 27, 29],
 [205, 33, 147, 9, 2, 17, 66],
 [3, 32, 36, 30, 16, 10, 9],
 [3, 1, 44, 46, 57, 171, 13, 84, 378],
 [1, 44, 46, 62],
 [20, 58, 1, 65, 11, 4, 6, 41, 1, 240],
 [33, 1, 9, 4, 6],
 [16, 1, 8, 11],
 [3, 13, 24, 31, 1],
 [1, 56, 139, 185, 60, 63],
 [28, 14, 187, 1, 8,

In [22]:
#We use pad_sequences from tensorflow.keras.preprocessing.sequence library
train_data= pad_sequences(tokenized_train_data, maxlen= 20, padding= "pre")
test_data= pad_sequences(tokenized_test_data, maxlen= 20, padding= "pre")

In [23]:
train_data

array([[  0,   0,   0, ...,   5, 376,  21],
       [  0,   0,   0, ...,  10,  45,  21],
       [  0,   0,   0, ...,   1,  12,  18],
       ...,
       [  0,   0,   0, ...,  13,  19,   5],
       [  0,   0,   0, ...,   6,  51,   5],
       [  0,   0,   0, ...,   5,   4,   6]], dtype=int32)

In [24]:
train_data.shape

(4834, 20)

In [25]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

trans_matrix_train= transform_matrix(train_data, text_tokenizer) #This will be the matrix on which the lstm model is applied
trans_matrix_test= transform_matrix(test_data, text_tokenizer) #This will be the matrix on which the model is tested

In [26]:
trans_matrix_train.shape

(4834, 20, 654)

In [27]:
trans_matrix_test.shape

(800, 20, 654)

## **Model Building**

In [28]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Input, LSTM, Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy as cce
from tensorflow.keras.metrics import AUC
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_uniform, glorot_uniform

def build_lstm_model(input_dimensions, op_shape, num_steps, dropout_rate, kernel_reg, bias_reg):
    ip_layer = Input(shape=(num_steps, input_dimensions))
    lstm_model = LSTM(units=num_steps)(ip_layer)
    dense_layer_1 = Dense(op_shape,
                       kernel_initializer=he_uniform(),
                       bias_initializer="zeros",
                       kernel_regularizer=l2(kernel_reg),
                       bias_regularizer=l2(bias_reg))(lstm_model)

    int_layer = BatchNormalization()(dense_layer_1)
    int_layer = Activation('relu')(int_layer)
    int_layer = Dropout(rate=dropout_rate)(int_layer)

    output_1 = Dense(op_shape,
                  kernel_initializer=glorot_uniform(),
                  bias_initializer="zeros",
                  kernel_regularizer=l2(kernel_reg),
                  bias_regularizer=l2(bias_reg))(dense_layer_1)

    output_1 = BatchNormalization()(output_1)
    final_output = Activation('softmax')(output_1)

    optimizer = Adam()

    final_model = Model(inputs=[ip_layer], outputs=[final_output])
    final_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['AUC'])

    return final_model
x = np.random.rand(100, num_steps, input_dimensions)  # Example input data
y = np.random.randint(0, 2, size=(100, op_shape))  # Example labels
valid_split = 0.2

# Define your variables
input_dimensions = 10
op_shape = 5
num_steps = 20
dropout_rate = 0.2
kernel_reg = 0.001
bias_reg = 0.001
epochs = 10  # for example

# Assuming you have x, y, valid_split, defined somewhere
model = build_lstm_model(input_dimensions, op_shape, num_steps, dropout_rate, kernel_reg, bias_reg)

history = model.fit(x, y, validation_split=valid_split, epochs=epochs)
print(history.history)  # or do whatever you want with history

predictions = model.predict(x)
print(predictions)

NameError: name 'num_steps' is not defined

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import AUC
# Extracting metrics from history object
loss = history.history['loss']
val_loss = history.history['val_loss']
auc = history.history['AUC']
val_auc = history.history['val_AUC']

epochs = range(1, len(loss) + 1)

# Plotting losses
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plotting accuracy
plt.plot(epochs, auc, 'b', label='Training AUC')
plt.plot(epochs, val_auc, 'r', label='Validation AUC')
plt.title('Training and validation AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()
plt.show()